In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 57 kB/s 
     |████████████████████████████████| 199 kB 55.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=892d54c48396c58c9f8a41b857a421dc10f91fff44864d52d3f91ac4d7920c17
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.appName('SparkWindowFunction').getOrCreate()

In [10]:
simpleData = (("James", "Sales", 3000), \
    ("Sumit", "Sales", 4600),  \
    ("Rohit", "Sales", 4100),   \
    ("Mitesh", "Finance", 3000),  \
    ("Gursheen", "Sales", 3000),    \
    ("Yash", "Finance", 3300),  \
    ("Ben", "Finance", 3900),    \
    ("Pnadey", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Shah", "Sales", 4100) \
  )

In [12]:
columns= ["employee_name", "department", "salary"]

In [13]:
df = spark.createDataFrame(data = simpleData, schema = columns)

In [14]:
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Sumit        |Sales     |4600  |
|Rohit        |Sales     |4100  |
|Mitesh       |Finance   |3000  |
|Gursheen     |Sales     |3000  |
|Yash         |Finance   |3300  |
|Ben          |Finance   |3900  |
|Pnadey       |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Shah         |Sales     |4100  |
+-------------+----------+------+



In [15]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [16]:
windowSpec  = Window.partitionBy("department").orderBy("salary")

In [18]:
df.withColumn("row_number",row_number().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|Mitesh       |Finance   |3000  |1         |
|Yash         |Finance   |3300  |2         |
|Ben          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Pnadey       |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|Gursheen     |Sales     |3000  |2         |
|Rohit        |Sales     |4100  |3         |
|Shah         |Sales     |4100  |4         |
|Sumit        |Sales     |4600  |5         |
+-------------+----------+------+----------+



In [19]:
from pyspark.sql.functions import rank
df.withColumn("rank",rank().over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|       Mitesh|   Finance|  3000|   1|
|         Yash|   Finance|  3300|   2|
|          Ben|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|       Pnadey| Marketing|  3000|   2|
|        James|     Sales|  3000|   1|
|     Gursheen|     Sales|  3000|   1|
|        Rohit|     Sales|  4100|   3|
|         Shah|     Sales|  4100|   3|
|        Sumit|     Sales|  4600|   5|
+-------------+----------+------+----+



In [20]:
from pyspark.sql.functions import dense_rank
df.withColumn("dense_rank",dense_rank().over(windowSpec)).show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|       Mitesh|   Finance|  3000|         1|
|         Yash|   Finance|  3300|         2|
|          Ben|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|       Pnadey| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|     Gursheen|     Sales|  3000|         1|
|        Rohit|     Sales|  4100|         2|
|         Shah|     Sales|  4100|         2|
|        Sumit|     Sales|  4600|         3|
+-------------+----------+------+----------+



In [21]:
from pyspark.sql.functions import percent_rank
df.withColumn("percent_rank",percent_rank().over(windowSpec)).show()

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|       Mitesh|   Finance|  3000|         0.0|
|         Yash|   Finance|  3300|         0.5|
|          Ben|   Finance|  3900|         1.0|
|        Kumar| Marketing|  2000|         0.0|
|       Pnadey| Marketing|  3000|         1.0|
|        James|     Sales|  3000|         0.0|
|     Gursheen|     Sales|  3000|         0.0|
|        Rohit|     Sales|  4100|         0.5|
|         Shah|     Sales|  4100|         0.5|
|        Sumit|     Sales|  4600|         1.0|
+-------------+----------+------+------------+



In [22]:
from pyspark.sql.functions import ntile
df.withColumn("ntile",ntile(2).over(windowSpec)).show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|       Mitesh|   Finance|  3000|    1|
|         Yash|   Finance|  3300|    1|
|          Ben|   Finance|  3900|    2|
|        Kumar| Marketing|  2000|    1|
|       Pnadey| Marketing|  3000|    2|
|        James|     Sales|  3000|    1|
|     Gursheen|     Sales|  3000|    1|
|        Rohit|     Sales|  4100|    1|
|         Shah|     Sales|  4100|    2|
|        Sumit|     Sales|  4600|    2|
+-------------+----------+------+-----+



In [23]:
from pyspark.sql.functions import cume_dist    
df.withColumn("cume_dist",cume_dist().over(windowSpec)).show()

+-------------+----------+------+------------------+
|employee_name|department|salary|         cume_dist|
+-------------+----------+------+------------------+
|       Mitesh|   Finance|  3000|0.3333333333333333|
|         Yash|   Finance|  3300|0.6666666666666666|
|          Ben|   Finance|  3900|               1.0|
|        Kumar| Marketing|  2000|               0.5|
|       Pnadey| Marketing|  3000|               1.0|
|        James|     Sales|  3000|               0.4|
|     Gursheen|     Sales|  3000|               0.4|
|        Rohit|     Sales|  4100|               0.8|
|         Shah|     Sales|  4100|               0.8|
|        Sumit|     Sales|  4600|               1.0|
+-------------+----------+------+------------------+



In [24]:
windowSpecAgg  = Window.partitionBy("department")


from pyspark.sql.functions import col,avg,sum,min,max,row_number 
df.withColumn("row",row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("department","avg","sum","min","max").show()

+----------+------+-----+----+----+
|department|   avg|  sum| min| max|
+----------+------+-----+----+----+
|   Finance|3400.0|10200|3000|3900|
| Marketing|2500.0| 5000|2000|3000|
|     Sales|3760.0|18800|3000|4600|
+----------+------+-----+----+----+

